In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp
import nnqs
import sampler

2025-07-09 10:51:13.088775: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752051073.111094   13183 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752051073.118255   13183 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752051073.135560   13183 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752051073.135579   13183 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752051073.135581   13183 computation_placer.cc:177] computation placer alr

In [2]:
n_spins = 16
# density = hidden/visible
density = 4
n_hidden_spins = int(n_spins * density)

wave = nnqs.RBM(n_visible=n_spins, n_hidden=n_hidden_spins)
mrt2_sampler = sampler.MRT2(n_sites=n_spins, batch_size=10, n_chains=1, n_sweeps=10)

print(mrt2_sampler.current_state)

<tf.Variable 'mrt2_state:0' shape=(1, 16) dtype=int32, numpy=array([[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]], dtype=int32)>


I0000 00:00:1752051076.451549   13183 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2609 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5


In [3]:
def target_log_prob(*state_parts):
    print("State parts:", state_parts)
    if len(state_parts) == 1:
        state = state_parts[0]
    else:
        state = state_parts[0]
    
    return wave.log_prob(state)


metro_kernel = tfp.mcmc.RandomWalkMetropolis(
    target_log_prob_fn=target_log_prob,
    new_state_fn=mrt2_sampler._single_site_flip
)

init_state = metro_kernel.bootstrap_results(mrt2_sampler.current_state)

print("Initial state:", init_state)

result = metro_kernel.one_step(
    current_state=mrt2_sampler.current_state,
    previous_kernel_results=init_state,
    seed=42
)

print("New state after one step:", result[0])
print("Kernel results:", result[1])


State parts: (<tf.Tensor: shape=(1, 16), dtype=int32, numpy=array([[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]], dtype=int32)>,)
Initial state: MetropolisHastingsKernelResults(
  accepted_results=UncalibratedRandomWalkResults(
      log_acceptance_correction=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
      target_log_prob=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([97.38903], dtype=float32)>,
      seed=[]
    ),
  is_accepted=<tf.Tensor: shape=(1,), dtype=bool, numpy=array([ True])>,
  log_accept_ratio=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
  proposed_state=<tf.Variable 'mrt2_state:0' shape=(1, 16) dtype=int32, numpy=array([[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]], dtype=int32)>,
  proposed_results=UncalibratedRandomWalkResults(
      log_acceptance_correction=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
      target_log_prob=<tf.Tensor: shape=(1,), dtype=float32, numpy

In [21]:
samples = mrt2_sampler.sample_with_diagnostics(wave)

print("Generated samples:")
print(samples)


Generated samples:
(<tf.Tensor: shape=(10, 16), dtype=float32, numpy=
array([[1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],
      dtype=float32)>, <tf.Tensor: shape=(), dtype=float32, numpy=0.699999988079071>, <tf.Tensor: shape=(10, 1), dtype=bool, numpy=
array([[ True],
       [False],
       [False],
       [ True],
   

In [11]:
print(mrt2_sampler.current_state)
print(wave.a)
print(wave.b)
print(wave.W)

wave.log_psi(mrt2_sampler.current_state)

<tf.Variable 'mrt2_state:0' shape=(1, 16) dtype=int32, numpy=array([[1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>
<tf.Variable 'visible_bias:0' shape=(16,) dtype=float32, numpy=
array([ 0.05182908, -0.03531402, -0.12881862, -0.11880983, -0.03450819,
        0.11554106, -0.05776259,  0.07570194,  0.04771845,  0.1346591 ,
       -0.07797218, -0.14376481,  0.08132702,  0.03087344,  0.00092941,
        0.07636075], dtype=float32)>
<tf.Variable 'hidden_bias:0' shape=(64,) dtype=float32, numpy=
array([ 0.0247687 ,  0.0779311 ,  0.01699164, -0.0206477 , -0.05960264,
       -0.0558875 ,  0.00685559,  0.04007961, -0.03883881, -0.14146566,
       -0.17871659, -0.30964136, -0.0067846 , -0.08259855, -0.18235411,
       -0.03261776,  0.2097867 ,  0.1591655 , -0.11178732,  0.04261604,
       -0.12745582, -0.0184547 , -0.02203484, -0.02758373,  0.06034005,
        0.0179116 ,  0.00816443, -0.02608932, -0.0705049 ,  0.03059687,
        0.19273375,  0.03730853, -0.18609823,  0.029306

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([46.73649], dtype=float32)>

In [6]:
wave_test = nnqs.RBM(n_visible=8, n_hidden=8)

states0 = tf.zeros((1, 8), dtype=tf.float32)  # All zeros
states1 = tf.ones((1, 8), dtype=tf.float32)   # All ones
states_r = tf.random.uniform((1, 8), maxval=2, dtype=tf.int32)
states_r = tf.cast(states_r, tf.float32)

print("Log prob (all zeros):", wave_test.log_prob(states0))
print("Log prob (all ones):", wave_test.log_prob(states1))  
print("Log prob (random):", wave_test.log_prob(states_r))
print("Prob (all zeros):", tf.exp(wave_test.log_prob(states0)))
print("Prob (all ones):", tf.exp(wave_test.log_prob(states1)))
print("Prob (random):", tf.exp(wave_test.log_prob(states_r)))

Log prob (all zeros): tf.Tensor([10.195515], shape=(1,), dtype=float32)
Log prob (all ones): tf.Tensor([12.828673], shape=(1,), dtype=float32)
Log prob (random): tf.Tensor([11.743578], shape=(1,), dtype=float32)
Prob (all zeros): tf.Tensor([26782.787], shape=(1,), dtype=float32)
Prob (all ones): tf.Tensor([372753.78], shape=(1,), dtype=float32)
Prob (random): tf.Tensor([125942.15], shape=(1,), dtype=float32)


In [7]:
test_states = tf.random.uniform((5, 8), maxval=2, dtype=tf.int32)
print("Test states:", test_states.numpy())
test_states = tf.cast(test_states, tf.float32)
print("Test states (as float):", test_states.numpy())
log_probs = wave_test.log_prob(test_states)
sum_visible = tf.reduce_sum(wave_test.a * test_states, axis=1)
sum_hidden = tf.reduce_sum(tf.math.log(2.0*(tf.math.cosh(wave_test.b + tf.matmul(test_states, wave_test.W)))), axis=1)
print("Sum of visible biases:", sum_visible.numpy())
print("Sum of hidden biases:", sum_hidden.numpy())
log_prob_test = sum_visible + sum_hidden
print("Log prob (manual):", log_prob_test.numpy())
print("Prob:", tf.exp(log_prob_test).numpy())

print("Log prob:", log_probs)
print("Prob:", tf.exp(log_probs))

Test states: [[1 0 0 1 1 0 1 0]
 [0 0 1 0 0 1 1 0]
 [1 1 0 1 1 0 0 0]
 [1 1 0 0 1 0 0 0]
 [1 1 0 1 0 1 1 1]]
Test states (as float): [[1. 0. 0. 1. 1. 0. 1. 0.]
 [0. 0. 1. 0. 0. 1. 1. 0.]
 [1. 1. 0. 1. 1. 0. 0. 0.]
 [1. 1. 0. 0. 1. 0. 0. 0.]
 [1. 1. 0. 1. 0. 1. 1. 1.]]
Sum of visible biases: [-0.01127508 -0.15618768  0.08018745 -0.01374663 -0.14962626]
Sum of hidden biases: [5.9655647 5.7794924 5.9144077 5.805585  5.9789953]
Log prob (manual): [5.9542894 5.623305  5.994595  5.791838  5.829369 ]
Prob: [385.40295 276.80267 401.25415 327.6147  340.14398]
Log prob: tf.Tensor([11.95346   9.993798 12.182076 12.037144 11.934568], shape=(5,), dtype=float32)
Prob: tf.Tensor([155353.7    21890.285 195257.89  168913.78  152446.4  ], shape=(5,), dtype=float32)


In [8]:
test_state = tf.constant([[1, 0, 1, 0, 1, 0, 1, 0]], dtype=tf.int32)
flipped = mrt2_sampler._single_site_flip(test_state)
print("Original:", test_state)
print("Log prob (Original)", wave_test.log_prob(test_state))
print("Flipped: ", flipped)
print("Log prob (Flipped)", wave_test.log_prob(flipped))
print("Ratio of log:", wave_test.log_prob(flipped) - wave_test.log_prob(test_state))

2025-07-09 10:51:20.224346: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,16] vs. [1,8]


InvalidArgumentError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} Incompatible shapes: [1,16] vs. [1,8] [Op:Mul] name: 